# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [ ]:
import pandas as pd
import logging
import json
from utilities import text_to_json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [ ]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()
        responses = scoring_func(inputString)
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Let's test the module.

In [ ]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [ ]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [ ]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

Here, we convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [ ]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [ ]:
predict_for = get_model_api()

In [ ]:
resp = predict_for(body)

Next, we move on to [building our docker image](03_Build_Image.ipynb).